<a href="https://colab.research.google.com/github/tateemma/Sentiment-Analysis-App-with-Twint-and-Streamlit/blob/main/Sentiment_analysis_Web_App_with_Twint_and_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Twitter Data Sentiment Analysis Web App with Streamlit and Twint

In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#update twint
!pip install --upgrade --user -e git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [ ]:
#install streamlit
!pip install streamlit

### Streamlit file

In [ ]:
%%writefile sentiment.py
import sys
#sys.path.append("twint/")
import twint
import textblob
import re
import streamlit as st
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt

# Set up TWINT config
c = twint.Config()

# Solve compatibility issues with notebooks and RunTime errors. You don't need this with streamlit!
#import nest_asyncio
#nest_asyncio.apply()

st.header('Sentiment Analysis App')

st.sidebar.header('Enter Search Term')

search_term = st.sidebar.text_input('Type search term')

c.Search = str(search_term)
# Custom output format
c.Format = "Username: {username} |  Tweet: {tweet}"
c.Limit = 1000
c.Pandas = True

twint.run.Search(c)

def available_columns():
    return twint.output.panda.Tweets_df.columns

def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]

df_pd = twint_to_pandas(["date", "username", "tweet", "hashtags", "nlikes"])

def clean_tweet(tweet): 
        ''' 
        Clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

def get_sentiment(tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = textblob.TextBlob(clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'

df_pd['sentiment'] = df_pd.tweet.apply(lambda twt: get_sentiment(twt))

#write percentages of tweets according to sentiments
positive = (len(df_pd[df_pd['sentiment'] == 'positive'])/len(df_pd['sentiment'])) * 100
st.sidebar.write(round(positive,2), '% Positive')

neutral = (len(df_pd[df_pd['sentiment'] == 'neutral'])/len(df_pd['sentiment'])) * 100
st.sidebar.write(round(neutral,2), '% Neutral')

negative = (len(df_pd[df_pd['sentiment'] == 'negative'])/len(df_pd['sentiment'])) * 100
st.sidebar.write(round(negative,2), '% Negative')

st.subheader('Pie Chart')
fig, ax = plt.subplots()
ax.pie([positive, negative, neutral], radius=0.7, autopct='%1.2f%%')
st.pyplot(fig)

st.subheader('Wordcloud')
cloud_tweets = df_pd.tweet.apply(lambda twt: clean_tweet(twt))
wordcloud = WordCloud(background_color = 'white', width = 800, height = 350,).generate(str(cloud_tweets))
st.image(wordcloud.to_array())


Overwriting sentiment.py


### Run with ngrok (for google colab)

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

In [ ]:
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://c9a51c99deaa.ngrok.io


In [ ]:
!streamlit run /content/sentiment.py